In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import time
import pandas as pd

import requests
import lxml
from lxml.html import fromstring

# headers
userAgent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
headers = {'User-Agent' : userAgent}

# option 
option = Options()
option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

# 파라미터: 검색하고자하는 단어

In [2]:
KEYWORDS = ['건전지', '축구공']

# URL 크롤링

In [3]:
# df 정의
df = pd.DataFrame(columns=['url','키워드','id'])
t1 = time.time()

    
for KEYWORD in KEYWORDS:

    # log
    print(f'== 키워드: {KEYWORD} 시작 == 경과시간: {round(time.time()-t1)}초')


    try:
        url = 'https://www.coupang.com/np/search?rocketAll=false&q={}&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&component=&rating=0&sorter=saleCountDesc&listSize=36'.format(KEYWORD)
        res = requests.get(url, headers=headers)
        parser = fromstring(res.text)

        # url명 크롤링
        url_list = []
        elements = parser.xpath('/html/body/div[2]/section/form/div[2]/div[2]/ul/li')
        for element in elements[:10]:
            url = 'https://www.coupang.com'+element.findall('a')[0].get('href')
            url_list.append(url)

        # 상품명 크롤링
        product_list = []
        products = parser.xpath('//a/dl/dd/div/div[2]')
        for product in products[:10]:
            product_list.append(product.text_content())

        tmp = pd.DataFrame(url_list, columns=['url'])
        tmp['상품명'] = product_list
        tmp['키워드'] = KEYWORD
        tmp['id'] = tmp['url'].apply(lambda x: x.split('/')[5].split('?')[0])
        df = df.append(tmp, ignore_index=True)

    except:
        print('1차에러!!!')
        url = 'https://www.coupang.com/np/search?rocketAll=false&q={}&brand=&offerCondition=&filter=&availableDeliveryFilter=&filterType=&isPriceRange=false&priceRange=&minPrice=&maxPrice=&page=1&trcid=&traid=&filterSetByUser=true&channel=user&backgroundColor=&component=&rating=0&sorter=saleCountDesc&listSize=36'.format(KEYWORD)
        res = requests.get(url, headers=headers)
        parser = fromstring(res.text)

        # url명 크롤링
        url_list = []
        elements = parser.xpath('/html/body/div[2]/section/form/div[2]/div[2]/ul/li')
        for element in elements[:10]:
            url = 'https://www.coupang.com'+element.findall('a')[0].get('href')
            url_list.append(url)

        # 상품명 크롤링
        product_list = []
        products = parser.xpath('//a/dl/dd/div/div[2]')
        for product in products[:10]:
            product_list.append(product.text_content())

        tmp = pd.DataFrame(url_list, columns=['url'])
        tmp['상품명'] = product_list
        tmp['키워드'] = KEYWORD
        tmp['id'] = tmp['url'].apply(lambda x: x.split('/')[5].split('?')[0])
        df = df.append(tmp, ignore_index=True)
            
print('========================== 크롤링완료! ==========================')
df = df.drop_duplicates(subset=['id']).reset_index(drop=True)
df

== 키워드: 건전지 시작 == 경과시간: 0초
== 키워드: 축구공 시작 == 경과시간: 1초
========================== 크롤링완료! ==========================


,url,키워드,id,상품명
0,https://www.coupang.com/vp/products/220853174?...,건전지,220853174,"시터스 알카라인 AA건전지, 32개입, 1박스"
1,https://www.coupang.com/vp/products/130164876?...,건전지,130164876,"듀라셀 알카라인 AA 건전지, 20개입, 1개"
2,https://www.coupang.com/vp/products/130164870?...,건전지,130164870,"듀라셀 알카라인 건전지 AA 20p + AAA 20p, 1세트"
3,https://www.coupang.com/vp/products/220853185?...,건전지,220853185,"시터스 알카라인 AAA건전지, 32개입, 1박스"
4,https://www.coupang.com/vp/products/307012900?...,건전지,307012900,"시터스 알카라인 건전지 패밀리팩 AA 20p + AAA 20p, 1세트"
5,https://www.coupang.com/vp/products/330633297?...,건전지,330633297,"에너자이저 알카라인 AA 건전지, 36개입, 1개"
6,https://www.coupang.com/vp/products/130164868?...,건전지,130164868,"듀라셀 알카라인 건전지 AAA, 20개입, 1개"
7,https://www.coupang.com/vp/products/4737370566...,건전지,4737370566,"벡셀 알카라인 AA건전지 벌크, 48개입, 1박스"
8,https://www.coupang.com/vp/products/32741820?i...,건전지,32741820,"벡셀 알카라인 AAA건전지, 48개입, 1개"
9,https://www.coupang.com/vp/products/37405451?i...,축구공,37405451,"비바 스피드 양방향 공기주입 펌프, 혼합 색상"


# 세부 값들 크롤링

In [5]:
result = pd.DataFrame(columns=['글쓴이','별점','날짜','상품명','제목','내용','url','순위'])
t1 = time.time()


for url_idx in range(0,df.shape[0]):

    
    #쿠키 / 캐쉬파일 삭제
    try:
        shutil.rmtree(r"c:\chrometemp")  
    except:
        pass


    # chrome.exe로 접근
    subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동


    # url 접근
    driver = webdriver.Chrome('.\\webdriver\\chromedriver.exe', options=option)
    url = df.loc[url_idx, 'url']
    
    try:        
        driver.get(url)
        
    except:
        time.sleep(3)
        driver.get(url)
        
    time.sleep(5)
        

    # <필수 표기정보> 까지 화면을 이동
    location = driver.find_element_by_xpath('//*[@id="itemBrief"]/div/p[1]')
    driver.execute_script(f"window.scrollTo(0, {location.location['y']})") 


    # <상품평> 클릭
    driver.find_element_by_xpath('//*[@id="btfTab"]/ul[1]/li[2]').click()
    time.sleep(2)

    
    # <별점보기 칸> 까지 이동
    location2 = driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[2]/div[1]')
    driver.execute_script(f"window.scrollTo(0, {location2.location['y']-300})")
    time.sleep(1)


    # 별점보기 클릭 (상품평이 없을수도 있음 -->  try, except)
    try:
        driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[2]/div[3]/div[1]/div[1]/img[2]').click()
        time.sleep(2)
    
    except:
        driver.close()
        print(f'=== {url_idx}번째 상품평이 없습니다. ===')
        continue


    # 나쁨, 별로, 보통 구분 motion
    motion1 = driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[2]/div[3]/div[2]/ul/li[5]')
    motion2 = driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[2]/div[3]/div[2]/ul/li[4]')
    motion3 = driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[2]/div[3]/div[2]/ul/li[3]')
    motions = [motion1, motion2, motion3]

    for motion in motions:

        # 페이지 초기화
        page_value = 3

        # 모션 클릭
        motion.click()
        time.sleep(2)


        while True:

            try:
                # 내용추출
                elements = driver.find_elements_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[4]/article')

                for element in elements:
                    name = element.find_elements_by_xpath('div[1]/div[2]/span')[0].text # 글쓴이
                    product = element.find_elements_by_xpath('div[1]/div[4]')[0].text # 상품명
                    date = element.find_elements_by_xpath('div[1]/div[3]/div[2]')[0].text # 날짜
                    star = element.find_elements_by_xpath('div[1]/div[3]/div[1]/div')[0].get_attribute('data-rating') # 별점

                    try:
                        headline = element.find_elements_by_class_name('sdp-review__article__list__headline')[0].text # 제목
                    except:
                        headline = ''

                    try:
                        content = element.find_elements_by_class_name('js_reviewArticleContent')[0].text # 내용
                    except:
                        content =  ''


                    # append result                   
                    tmp = pd.DataFrame({'글쓴이':[name], '별점':[star], '날짜':[date], '상품명':[product], '제목':[headline], '내용':[content], 'url':[url], '순위':[url_idx+1]}) # 본문추가하기
                    result = result.append(tmp, ignore_index=True)


                # 10개 채우면 초기화
                if page_value == 13:
                    page_value = 3
                    
                    
                # page 이동
                location_area=driver.find_element_by_xpath(f'//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[4]/div[3]/button[{page_value}]')
                driver.execute_script(f"window.scrollTo(0, {location_area.location['y']-200})") 
                time.sleep(1)
                location_area.click()
                time.sleep(2)

                
                # page += 1
                page_value += 1



            except: 
                break


        # while문이 종료되면 보통 -> 나쁨으로 바뀜
        driver.execute_script(f"window.scrollTo(0, {location2.location['y']-300})") 
        time.sleep(2)
        driver.find_element_by_xpath('//*[@id="btfTab"]/ul[2]/li[2]/div/div[5]/section[2]/div[3]/div[1]').click()
    
    # url complete log
    print(f'== {url_idx}번째 완료 == 경과시간: {round(time.time()-t1)}초')
    driver.close()
    
    # 100개 단위 중간저장
    if url_idx % 100 == 0:
        result.to_csv(f'중간결과_{url_idx}.csv)', index=False)
        print(f'== {url_idx}개 중간저장 완료 ==')

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다

In [ ]:
result